In [ ]:
import pickle

import geneformer.perturber_utils as pu
from datasets import load_from_disk
from geneformer import TOKEN_DICTIONARY_FILE
from torch.utils.tensorboard import SummaryWriter
from transformers import AutoTokenizer

from lib.data_manager import perturb
from lib.modules import DREAMSEncoder

writer = SummaryWriter()

In [2]:

# format to specify perturbations
perturbations = {
    'ENSG00000066427' : 'overexpress',
    'ENSG00000066084':'delete'
}

# code to load a tokenized dataset and encode it
# should go to data manager class


dataset = load_from_disk('data/example_data')

with open(TOKEN_DICTIONARY_FILE, "rb") as f:
    gene_token_dict = pickle.load(f)

dataset.set_format('torch')
subset = dataset[0:8]
input_data_minibatch = subset["input_ids"]
max_len = int(max(subset["length"]))
pad_token_id = gene_token_dict.get("<pad>")
model_input_size = 2048

input_data_minibatch = pu.pad_tensor_list(
    input_data_minibatch, max_len, pad_token_id, model_input_size
)

perturbed = perturb(subset,perturbations, gene_token_dict)
perturbed_minibatch = pu.pad_tensor_list(
    input_data_minibatch, max_len, pad_token_id, model_input_size )

"""with torch.no_grad():
    outputs = model( input_data_minibatch.to("cuda"),
                     pu.gen_attention_mask(subset),
                     perturbed_minibatch.to("cuda"),
                     pu.gen_attention_mask(perturbed),
                     )
toplot = PCA(n_components=2).fit_transform(outputs.cpu().detach().numpy())
del model
torch.cuda.empty_cache()
plt.scatter(toplot[:,0], toplot[:,1])
"""


'with torch.no_grad():\n    outputs = model( input_data_minibatch.to("cuda"),\n                     pu.gen_attention_mask(subset),\n                     perturbed_minibatch.to("cuda"),\n                     pu.gen_attention_mask(perturbed),\n                     )\ntoplot = PCA(n_components=2).fit_transform(outputs.cpu().detach().numpy())\ndel model\ntorch.cuda.empty_cache()\nplt.scatter(toplot[:,0], toplot[:,1])\n'

In [3]:
tokenizer = AutoTokenizer.from_pretrained("ibm/MoLFormer-XL-both-10pct", trust_remote_code=True)

smiles = ["Cn1c(=O)c2c(ncn2C)n(C)c1=O", "CC(=O)Oc1ccccc1C(=O)O"]
smile_inputs = tokenizer(smiles, padding=True, return_tensors="pt")
"""with torch.no_grad():
    outputs = model(**inputs)"""

'with torch.no_grad():\n    outputs = model(**inputs)'

In [4]:
model = DREAMSEncoder()

In [5]:
model({ 'x':input_data_minibatch.to("cuda"),
        'x_attn':pu.gen_attention_mask(subset),
        'y': perturbed_minibatch.to("cuda"),
        'y_attn': pu.gen_attention_mask(perturbed),
        },
      smile_inputs.to('cuda')
      )

(tensor([[ 0.0420, -0.0235, -0.0217,  ..., -0.0214, -0.0529, -0.0199],
         [ 0.0420, -0.0235, -0.0217,  ..., -0.0214, -0.0529, -0.0199],
         [ 0.0420, -0.0235, -0.0217,  ..., -0.0214, -0.0529, -0.0199],
         ...,
         [ 0.0420, -0.0235, -0.0217,  ..., -0.0214, -0.0529, -0.0199],
         [ 0.0420, -0.0235, -0.0217,  ..., -0.0214, -0.0529, -0.0199],
         [ 0.0420, -0.0235, -0.0217,  ..., -0.0214, -0.0529, -0.0199]],
        device='cuda:0', grad_fn=<AddmmBackward0>),
 tensor([[-3.4558e-02, -6.0025e-02, -1.4580e-01, -7.5377e-02, -1.0751e+00,
          -1.9703e-01,  2.0103e-01, -2.8487e-01, -6.1164e-02,  1.5493e-01,
          -1.9034e-01,  3.4944e-01,  4.3032e-01,  1.8438e-01,  3.7300e-01,
          -3.0049e-01,  9.9008e-02,  1.4720e-01,  3.6196e-01, -5.0619e-02,
          -3.3424e-01, -2.5133e-01, -8.5994e-02, -1.2848e-01,  1.5987e-01,
          -7.3626e-02, -4.0117e-02,  5.0431e-01, -3.9033e-01,  2.5837e-01,
           3.2908e-01,  4.5762e-02,  7.7540e-02, -5.1747e

In [18]:
smile_inputs = smile_inputs.to('cuda')
writer.add_graph(model, ({ 'x':input_data_minibatch.to("cuda"),
                           'x_attn':pu.gen_attention_mask(subset),
                           'y': perturbed_minibatch.to("cuda"),
                           'y_attn': pu.gen_attention_mask(perturbed),
                           },
                         {
                             'input_ids':smile_inputs['input_ids'],
                             'attention_mask':smile_inputs['attention_mask']
                         }))


FileNotFoundError: [Errno 2] No such file or directory: b'runs/Apr03_16-00-42_JerkOS/events.out.tfevents.1743688842.JerkOS.8314.0'